In [1]:
from sklearn import metrics
!pip install scipy numpy matplotlib pandas sklearn tabulate seaborn folium geopy geopandas requests> /dev/null
%matplotlib notebook

In [2]:
# import importlib
# import sys
# importlib.reload(sys.modules['statistics.company'])
# Load libraries
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
from matplotlib import pyplot
from sklearn.model_selection import train_test_split, learning_curve, permutation_test_score
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import StratifiedKFold
from sklearn.preprocessing import normalize
from sklearn.metrics import classification_report, roc_auc_score, roc_curve, plot_roc_curve, auc
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score, precision_score, recall_score
from sklearn.metrics import plot_confusion_matrix
from sklearn.metrics import calinski_harabasz_score, davies_bouldin_score, silhouette_score
from sklearn.cluster import KMeans
from sklearn.tree import DecisionTreeClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import SVC
from sklearn.neural_network import MLPClassifier
from sklearn.preprocessing import LabelEncoder, OneHotEncoder
from tabulate import tabulate
from IPython.display import display, HTML
display(HTML("<style>.container { width:90% !important; }</style>"))
import os
import sys
sys.path.insert(0, os.path.abspath('/home/jovyan/host-note/module-dir'))

In [3]:
from dataset_provider.provider import get_raw_dataset
from dataset_provider.config import create_global_config
df = get_raw_dataset(create_global_config())

GetRawDataset - Started
GetRawDataset - read_json - DONE
GetRawDataset - map_columns_to_csv - DONE
GetRawDataset - read_csv - DONE
Found 304 job ads without salary range or currency
(1249, 840)
        salary_from     salary_to
count   1249.000000   1249.000000
mean   11148.760608  15884.851081
std     4540.909690   5114.312359
min     1250.000000   1750.000000
25%     8000.000000  12100.000000
50%    10000.000000  16000.000000
75%    14000.000000  19000.000000
max    44000.000000  50000.000000
Found 8 countries and 4 currencies!
Dropping foreign countries and translating currencies...
Unique countries: 1, currencies: 1, observations: 1230


/opt/conda/lib/python3.7/site-packages/pandas/core/indexing.py:966: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self.obj[item] = s


---------------------------------------------------------------------------

# Regression

In [4]:
print(df.columns)
def move_col_to_end(df, col):
    cols_at_end = [col]
    return df[[c for c in df if c not in cols_at_end] 
            + [c for c in cols_at_end if c in df]]

Index(['Unnamed: 0', 'title', 'city', 'country_code', 'marker_icon', 'remote',
       'company_name', 'company_size', 'experience_level', 'salary_from',
       ...
       'test lead', 'typoscript', 'unit tests', 'vendor management', 'vert',
       'web/network security', 'workshop', 'yii2', 'środowisko JAVA', 'С#'],
      dtype='object', length=840)


In [5]:
cl_df = df.copy()
for col in [ "Unnamed: 0", "title", "company_size", "country_code", "marker_icon", "company_name",
            "latitude", "longitude", "salary_currency", "published_at", "remote_interview", "id", "skills"]:
                cl_df=cl_df.drop(col, axis=1)
                
cl_df.columns


Index(['city', 'remote', 'experience_level', 'salary_from', 'salary_to',
       'employment_type', '.NET C#', '.NET Core', '.NET Core (5 stars)',
       '.NET Framework 4.5.1',
       ...
       'test lead', 'typoscript', 'unit tests', 'vendor management', 'vert',
       'web/network security', 'workshop', 'yii2', 'środowisko JAVA', 'С#'],
      dtype='object', length=827)

In [ ]:
print(cl_df.shape)

records = []
for i in range(0, cl_df.shape[0]):
    print(i)
    records.append([str(cl_df.values[i,j]) for j in range(0,  cl_df.shape[1])])

# records = np.zeros(cl_df.shape, str)
# for i in range(0, cl_df.shape[0]):
#     print(i)
#     for j in  range(0,  cl_df.shape[1]):
#         records[i][j] = str(cl_df.values[i,j])

(1230, 827)
0
1
2
3
4


In [ ]:
print(records)

In [ ]:
!pip install apriori
from apyori import apriori

association_rules = apriori(records, min_support=0.0045, min_confidence=0.2, min_lift=3, min_length=2)
association_results = list(association_rules)
